In [1]:
# !apt-get update && apt-get install wget -y

In [2]:
#!apt-get update && apt-get install libopenexr-dev -y
# !python3 -m pip install mediapy einshape chex

In [3]:
# !git clone https://github.com/deepmind/tapnet.git

In [4]:


# !cd tapnet/tapnet && pip3 install -r requirements.txt

In [28]:
# !export PYTHONPATH=`(cd ./ && pwd)`:`pwd`:$PYTHONPATH

# Main

In [1]:
from multiprocessing import Process
from multiprocessing import Event as PEvent
from multiprocessing import Queue as PQueue
import cv2 
from time import sleep, time
class Camera():

    def __init__(self, cap_id):
        self._cap = cv2.VideoCapture(cap_id)
        assert self._cap.isOpened(), f'can not connect to camera with id {cap_id}'
        

    def run(self, stop_event, outque):
        time_out = time()
        print('start polling frames')
        while not stop_event.is_set():
            try:
                ret, frame = self._cap.read()
                outque.put([frame, time()])
                #print(time() - time_out, end = '        \r')
                if outque.full():
                    frame_out, time_out = outque.get()
            except KeyboardInterrupt:
                pass
             
        self._cap.release()
        print()
        print(f'frames polling stopped               {not self._cap.isOpened()}',)
vidstop = PEvent()
vidque = PQueue(2)
vcap = Camera(0)
video_poll_process = Process(target=vcap.run, args=(vidstop, vidque, ))
video_poll_process.start()

start polling frames


In [2]:
from math import sqrt, atan2, pi
import math
from adafruit_bno08x.i2c import BNO08X_I2C
from adafruit_bno08x import BNO_REPORT_LINEAR_ACCELERATION, BNO_REPORT_ACCELEROMETER, BNO_REPORT_GYROSCOPE, BNO_REPORT_MAGNETOMETER, BNO_REPORT_GRAVITY, BNO_REPORT_ROTATION_VECTOR
import busio
def find_roll(dqw, dqx, dqy, dqz):
    norm = sqrt(dqw * dqw + dqx * dqx + dqy * dqy + dqz * dqz)
    dqw = dqw / norm
    dqx = dqx / norm
    dqy = dqy / norm
    dqz = dqz / norm
    sinr_cosp = +2.0 * (dqw * dqx + dqy * dqz)
    cosr_cosp = +1.0 - 2.0 * (dqx * dqx + dqy * dqy)
    roll_raw = atan2(sinr_cosp, cosr_cosp)
    roll = roll_raw * 180.0 / pi
    if roll > 0:
        roll = 360 - roll
    else:
        roll = abs(roll)
    return roll

def find_pitch(dqw, dqx, dqy, dqz):
    norm = sqrt(dqw * dqw + dqx * dqx + dqy * dqy + dqz * dqz)
    dqw = dqw / norm
    dqx = dqx / norm
    dqy = dqy / norm
    dqz = dqz / norm
    sinp = sqrt(+1.0 + 2.0 * (dqw * dqy - dqx * dqz))
    cosp = sqrt(+1.0 - 2.0 * (dqw * dqy - dqx * dqz))
    pitch_raw = +2.0 * atan2(sinp, cosp) - (pi / 2.0)
    pitch = pitch_raw * 180.0 / pi
    if pitch > 0:
        pitch = 360 - pitch
    else:
        pitch = abs(pitch)
    return pitch

i2c = busio.I2C((1, 14), (1, 15))
imu_device = BNO08X_I2C(i2c, address=0x4b)
for feature in [BNO_REPORT_ACCELEROMETER, BNO_REPORT_GYROSCOPE, BNO_REPORT_MAGNETOMETER, BNO_REPORT_ROTATION_VECTOR]:
    imu_device.enable_feature(feature)
    
imu_stop = PEvent()
imu_que = PQueue(2)

def run_imu(imu_device, ev, que):
    time_out = time()
    print('start polling imu values')
    while not ev.is_set():
        try:
            q = imu_device.quaternion
            que.put([q, time()])
            if que.full():
                q, time_out = que.get()
        except KeyboardInterrupt:
            pass
    print("imu polling stopped")

imu_poll_process = Process(target=run_imu, args=(imu_device, imu_stop, imu_que,))
imu_poll_process.start()

start polling imu values


In [3]:
import matplotlib.pyplot as plt
import mediapy as media
import numpy as np
from PIL import Image

import torch
import torch.nn.functional as F
import torchvision.transforms as T

from tapnet.torch import tapir_model
from tapnet.utils import transforms
from tapnet.utils import viz_utils

import cv2 
import numpy as np
from time import sleep, time

from IPython.display import clear_output

def postprocess_occlusions(occlusions, expected_dist):
  visibles = (1 - F.sigmoid(occlusions)) * (1 - F.sigmoid(expected_dist)) > 0.5
  return visibles

def preprocess_frames(frames):
  """Preprocess frames to model inputs.

  Args:
    frames: [num_frames, height, width, 3], [0, 255], np.uint8

  Returns:
    frames: [num_frames, height, width, 3], [-1, 1], np.float32
  """
  frames = frames.float()
  frames = frames / 255 * 2 - 1
  return frames


def transform(frame):
    frame =  cv2.resize(frame, (fsize , fsize))
    tframe = torch.tensor(frame).permute(2,0,1)
    crop = tframe.float().permute(1,2,0)
    # crop = T.CenterCrop(fsize)(tframe).float().permute(1,2,0)
    return crop / 255 * 2 - 1

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

device = torch.device('cpu')
model = tapir_model.TAPIR(pyramid_level=1, initial_resolution = (64, 64))

model.load_state_dict(torch.load('tapnet/bootstapir_checkpoint.pt'))
model = model.to(device)

# SIFT

In [45]:
import cv2  as cv

CaM = np.array([[628.07730444,   0.        , 296.22347846],
       [  0.        , 624.87736912, 280.1399009 ],
       [  0.        ,   0.        ,   1.        ]])


tcrop = None
from IPython.display import display as IPdisplay, Image as IPImage

display_handle=IPdisplay(None, display_id=True)
tdpp = None

dpp_new = np.array([0, 0])
# first_frame_flag = True
try:
    while True:
        tic = time()
        cnt+=1
        frame, _ = vidque.get()
        q, tic_r = imu_que.get()
        
        tan_x = np.tan(-math.radians(find_pitch(q[3], q[0], q[1], q[2])))
        tan_y = np.tan(-math.radians(find_roll(q[3], q[0], q[1], q[2])))
        dpp = np.asarray([
            tan_x*CaM[0,0] + CaM[0,2],
            tan_y*CaM[1,1] + CaM[1,2],
        ])

        cv2.circle(frame, dpp.astype(np.int32).tolist(), 8, (255, 0, 0), -1)
        crop = frame        
        
        if tcrop is None:
            tframe = frame
            tcrop = crop
        if tdpp is None:
            tdpp = dpp
            tdpp = np.array([tdpp])

        color_point = (0, 255, 0)

        # Initiate SIFT detector
        sift = cv2.SIFT_create() 
         
        # find the keypoints and descriptors with SIFT
        kp1, des1 = sift.detectAndCompute(cv2.cvtColor(tframe, cv2.COLOR_BGR2GRAY),None)
        kp2, des2 = sift.detectAndCompute(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY),None)
         
        flann = cv.FlannBasedMatcher(dict(algorithm = 0, trees = 5), dict())

        if(len(kp1)>=2 and len(kp2)>=2):
            matches = flann.knnMatch(des1,des2,k=2)
        else:
            color_point = (255, 0, 0)
            matches = None
         
        # store all the good matches as per Lowe's ratio test.
        good = []
        if matches is not None:
            for m,n in matches:
                if m.distance < 0.7*n.distance:
                    good.append(m)
            src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
            dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        else:
            dst_pts = []

        if (len(dst_pts) >=4):
            matrix = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
        else:
            matrix = None
        
        # print(matrix[0])
        
        # matrix = cv2.findHomography(tracks[:,0,:].numpy().reshape(-1,1,2), tracks[:,1,:].numpy().reshape(-1,1,2), cv2.RANSAC, 5.0)[0]


        # np.linalg.inv(matrix)
        if matrix is not None and matrix[0] is not None:
            dpp_new = cv2.perspectiveTransform(tdpp.reshape(-1,1,2).astype(np.float32), matrix[0])
            dpp_new = dpp_new[0][0]
        else:
            color_point = (255, 0, 0)
            dpp_new[0]-=640
        shifts = dpp_new - dpp
 
    
        crops = []
        for img in [tcrop, crop]:
            # img = ((img.numpy() + 1) * 255 / 2).astype(np.uint8)
            crops.append(img)
    


        for index_, crop in enumerate(crops):
            crops[index_] = crop
        canvas = np.hstack(crops)
        

        frame = np.ndarray.astype(canvas[...,::-1], np.float32)
        frame = cv2.putText(frame, f"{1/(time()-tic):.2f}", (50, 50) , cv2.FONT_HERSHEY_SIMPLEX ,  
                           1, (255, 0, 0), 2, cv2.LINE_AA)
        frame = cv2.putText(frame, f"{shifts[0]:.2f},{shifts[1]:.2f}", (50, 200) , cv2.FONT_HERSHEY_SIMPLEX ,  
                           1, (0, 0, 255), 2, cv2.LINE_AA)
        
        dpp_new[0]+=640
        # print(dpp_new)
        cv2.circle(frame, dpp_new.astype(np.int32).tolist(), 8, color_point, -1)
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(IPImage(data=frame.tobytes()))
        
        # print(f'{1/(time()-tic):.2f}', end='          \r')
except KeyboardInterrupt:
    pass
finally:
    display_handle.update(None)
#Image.fromarray(canvas[...,::-1])

None

# TAPIR one point

In [6]:
fsize = 160
step = 8

CaM = np.array([[628.07730444,   0.        , 296.22347846],
       [  0.        , 624.87736912, 280.1399009 ],
       [  0.        ,   0.        ,   1.        ]])

points = np.asarray([[0, i+step//2, j+step//2] for i in range(0, fsize, step) for j in range(0, fsize, step)])
points = torch.tensor(points)[None].float().to(device)

tcrop = None
from IPython.display import display as IPdisplay, Image as IPImage

display_handle=IPdisplay(None, display_id=True)
tdpp = None

cnt = 0
try:
    while True:
        tic = time()
        cnt+=1
        frame, _ = vidque.get()
        q, tic_r = imu_que.get()
        
        tan_x = np.tan(-math.radians(find_pitch(q[3], q[0], q[1], q[2])))
        tan_y = np.tan(-math.radians(find_roll(q[3], q[0], q[1], q[2])))
        dpp = np.asarray([
            tan_x*CaM[0,0] + CaM[0,2],
            tan_y*CaM[1,1] + CaM[1,2],
        ])

        cv2.circle(frame, dpp.astype(np.int32).tolist(), 8, (255, 0, 0), -1)
        crop = transform(frame)
        dpp[0] *= (fsize/640)
        dpp[1] *= (fsize/480)
 
              
        
        if tcrop is None:
            tframe = frame
            tcrop = crop
        if tdpp is None:
            tdpp = dpp
            points = [[0., tdpp[1], tdpp[0]]]
            points = torch.tensor(points)[None].float().to(device)
            tdpp = np.array([tdpp])


        seq = torch.cat((tcrop[None], crop[None]))[None]
    
        outputs = model(seq, points)
        tracks, occlusions, expected_dist = outputs['tracks'][0], outputs['occlusion'][0], outputs['expected_dist'][0]
        # print((tracks[0][1]).numpy())
        # matrix = cv2.findHomography(tracks[:,0,:].numpy().reshape(-1,1,2), tracks[:,1,:].numpy().reshape(-1,1,2), cv2.RANSAC, 5.0)[0]

        # dpp_new = cv2.perspectiveTransform(tdpp.reshape(-1,1,2).astype(np.float32), matrix)

                
        dpp_new = (tracks[0][1]).numpy()
        # dpp_new = dpp_new[0][0]
        shifts = dpp_new - dpp
        shifts[0]*=(640/fsize)
        shifts[1]*=(480/fsize)
        print(shifts)

        crops = []
        for img in [tcrop, crop]:
            img = ((img.numpy() + 1) * 255 / 2).astype(np.uint8)
            crops.append(img)
        for pts in tracks:
            color = np.random.randint(low=0, high=255,size=3).tolist()
            for pt, img in zip(pts, crops):
                cv2.circle(img, pt.numpy().astype(np.int32).tolist(), 1, color, -1)

        for index_, crop in enumerate(crops):
            crop = cv2.resize(crop, (640, 480))
            crops[index_] = crop
        canvas = np.hstack(crops)
        

        frame = np.ndarray.astype(canvas[...,::-1], np.float32)
        frame = cv2.putText(frame, f"{1/(time()-tic):.2f}", (50, 50) , cv2.FONT_HERSHEY_SIMPLEX ,  
                           1, (255, 0, 0), 2, cv2.LINE_AA)
        
        dpp_new[0]*=(640/fsize)
        dpp_new[1]*=(480/fsize)
        dpp_new[0]+=640
        cv2.circle(frame, dpp_new.astype(np.int32).tolist(), 8, (0, 255, 0), -1)
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(IPImage(data=frame.tobytes()))
        
        if cnt > 19:
            print("OVER")
        if cnt > 20:
            cnt = 0
            clear_output(wait=True)
            display_handle=IPdisplay(None, display_id=True)
except KeyboardInterrupt:
    pass
finally:
    display_handle.update(None)

None

[-2.43046142  8.13807492]
[34.30026961 14.60707821]
[21.74726088 56.3335962 ]
[ -7.68830708 106.0557507 ]
[ 6.03629487 34.50278435]
[-43.00403011  69.15985251]
[ 6.79189008 15.27921557]
[-22.55337617  64.60778836]
[-45.91546014 112.26545663]
[-39.48814148  36.13306947]
[-8.38055149 -9.62078896]
[ 46.26659857 -25.82812456]
[  86.40590676 -111.31186312]
[ 88.59746652 -46.73912761]
[113.5342484  -54.86136432]


# TAPIR homography


In [6]:
fsize = 64#256
step = 8#64*4

CaM = np.array([[628.07730444,   0.        , 296.22347846],
       [  0.        , 624.87736912, 280.1399009 ],
       [  0.        ,   0.        ,   1.        ]])

points = np.asarray([[0, i+step//2, j+step//2] for i in range(0, fsize, step) for j in range(0, fsize, step)])
points = torch.tensor(points)[None].float().to(device)

tcrop = None
from IPython.display import display as IPdisplay, Image as IPImage

display_handle=IPdisplay(None, display_id=True)
tdpp = None

cnt = 0
# first_frame_flag = True
try:
    while True:
        tic = time()
        cnt+=1
        # ret, frame = cap.read()
        frame, _ = vidque.get()
        q, tic_r = imu_que.get()
        
        tan_x = np.tan(-math.radians(find_pitch(q[3], q[0], q[1], q[2])))
        tan_y = np.tan(-math.radians(find_roll(q[3], q[0], q[1], q[2])))
        dpp = np.asarray([
            tan_x*CaM[0,0] + CaM[0,2],
            tan_y*CaM[1,1] + CaM[1,2],
        ])

        cv2.circle(frame, dpp.astype(np.int32).tolist(), 8, (255, 0, 0), -1)
        crop = transform(frame)
        dpp[0] *= (fsize/640)
        dpp[1] *= (fsize/480)
        
        
        
        if tcrop is None:
            tframe = frame
            tcrop = crop
        if tdpp is None:
            tdpp = dpp
            tdpp = np.array([tdpp])
    
        seq = torch.cat((tcrop[None], crop[None]))[None]
    
        outputs = model(seq, points)
        tracks, occlusions, expected_dist = outputs['tracks'][0], outputs['occlusion'][0], outputs['expected_dist'][0]
        matrix = cv2.findHomography(tracks[:,0,:].numpy().reshape(-1,1,2), tracks[:,1,:].numpy().reshape(-1,1,2), cv2.RANSAC, 5.0)[0]


        # np.linalg.inv(matrix)
        dpp_new = cv2.perspectiveTransform(tdpp.reshape(-1,1,2).astype(np.float32), matrix)
        dpp_new = dpp_new[0][0]
        shifts = dpp_new - dpp
        shifts[0]*=(640/fsize)
        shifts[1]*=(480/fsize)
        print(shifts)
        # dpp[0] /= (fsize/480)
        # dpp[1] /= (fsize/640)
        
        # crop = transform(frame)
        
        
        # print(dpp, np.dot(np.array(np.append(dpp, 0)), matrix))
        # dpp = np.dot(matrix, np.array(np.append(dpp, 0)))


        # rotated = np.dot(np.linalg.inv(matrix), np.array(np.append(dpp, 0)))
        # print(rotated[:2] - tdpp)
 
    
        crops = []
        for img in [tcrop, crop]:
            img = ((img.numpy() + 1) * 255 / 2).astype(np.uint8)
            crops.append(img)
    
    
        # for pts in tracks:
        #     color = np.random.randint(low=0, high=255,size=3).tolist()
        #     for pt, img in zip(pts, crops):
        #         cv2.circle(img, pt.numpy().astype(np.int32).tolist(), 1, color, -1)
        """
        tcrop = crop
        
        tframe = frame

        tdpp = dpp
        """
        #viz

        for index_, crop in enumerate(crops):
            crop = cv2.resize(crop, (640, 480))
            crops[index_] = crop
        canvas = np.hstack(crops)
        

        frame = np.ndarray.astype(canvas[...,::-1], np.float32)
        frame = cv2.putText(frame, f"{1/(time()-tic):.2f}", (50, 50) , cv2.FONT_HERSHEY_SIMPLEX ,  
                           1, (255, 0, 0), 2, cv2.LINE_AA)
        
        dpp_new[0]*=(640/fsize)
        dpp_new[1]*=(480/fsize)
        dpp_new[0]+=640
        # print(dpp_new)
        cv2.circle(frame, dpp_new.astype(np.int32).tolist(), 8, (0, 255, 0), -1)
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(IPImage(data=frame.tobytes()))
        
        # print(f'{1/(time()-tic):.2f}', end='          \r')
        if cnt > 19:
            print("OVER")
        if cnt > 20:
            cnt = 0
            clear_output(wait=True)
            display_handle=IPdisplay(None, display_id=True)
except KeyboardInterrupt:
    pass
finally:
    display_handle.update(None)
#Image.fromarray(canvas[...,::-1])

None

[ 32.31164606 -42.05538609]
[  8.61859008 114.7506734 ]
[ 76.94139724 -29.16655895]
[ 60.06923469 -23.10683761]
[89.94264626 -2.10244784]
[63.95981521 57.29020157]
[99.70249814 30.0022021 ]
[63.06241207  6.38688062]
[76.04340493 53.93290018]
[62.49003813 52.18867038]
[69.85384771 44.07788093]
[27.44661738 53.97820926]
[84.79117535  1.40461785]
[54.12595043 68.9701967 ]
[91.68241022 40.45695344]
[91.05444998 29.7395854 ]
[100.95083291  -2.24109254]
[82.82016772 51.98131021]
[51.81215457 50.36100391]
[16.27251171 61.07455409]
OVER


# Two adjacent

In [25]:
fsize = 64#256
step = 8#64*4

CaM = np.array([[628.07730444,   0.        , 296.22347846],
       [  0.        , 624.87736912, 280.1399009 ],
       [  0.        ,   0.        ,   1.        ]])

points = np.asarray([[0, i+step//2, j+step//2] for i in range(0, fsize, step) for j in range(0, fsize, step)])
points = torch.tensor(points)[None].float().to(device)

tcrop = None
from IPython.display import display as IPdisplay, Image as IPImage

display_handle=IPdisplay(None, display_id=True)
tdpp = None

cnt = 0
try:
    while True:
        tic = time()
        cnt+=1
        # ret, frame = cap.read()
        frame, _ = vidque.get()
        q, tic_r = imu_que.get()
        
        tan_x = np.tan(-math.radians(find_pitch(q[3], q[0], q[1], q[2])))
        tan_y = np.tan(-math.radians(find_roll(q[3], q[0], q[1], q[2])))
        dpp = np.asarray([
            tan_x*CaM[0,0] + CaM[0,2],
            tan_y*CaM[1,1] + CaM[1,2],
        ])

        cv2.circle(frame, dpp.astype(np.int32).tolist(), 8, (255, 0, 0), -1)
        crop = transform(frame)
        dpp[0] *= (fsize/640)
        dpp[1] *= (fsize/480)
        
        
        
        if tcrop is None:
            tframe = frame
            tcrop = crop
        if tdpp is None:
            tdpp = dpp
    
        seq = torch.cat((tcrop[None], crop[None]))[None]
    
        outputs = model(seq, points)
        tracks, occlusions, expected_dist = outputs['tracks'][0], outputs['occlusion'][0], outputs['expected_dist'][0]
        matrix = cv2.findHomography(tracks[:,0,:].numpy().reshape(-1,1,2), tracks[:,1,:].numpy().reshape(-1,1,2), cv2.RANSAC, 5.0)[0]

        # print(tdpp)
        tdpp = np.array([tdpp])
        # np.linalg.inv(matrix)
        dpp_new = cv2.perspectiveTransform(tdpp.reshape(-1,1,2).astype(np.float32), matrix)
        dpp_new = dpp_new[0][0]
        print(dpp_new - dpp)
        # dpp[0] /= (fsize/480)
        # dpp[1] /= (fsize/640)
        
        # crop = transform(frame)
        
        
        # print(dpp, np.dot(np.array(np.append(dpp, 0)), matrix))
        # dpp = np.dot(matrix, np.array(np.append(dpp, 0)))


        # rotated = np.dot(np.linalg.inv(matrix), np.array(np.append(dpp, 0)))
        # print(rotated[:2] - tdpp)
 
    
        crops = []
        for img in [tcrop, crop]:
            img = ((img.numpy() + 1) * 255 / 2).astype(np.uint8)
            crops.append(img)
    
    
        # for pts in tracks:
        #     color = np.random.randint(low=0, high=255,size=3).tolist()
        #     for pt, img in zip(pts, crops):
        #         cv2.circle(img, pt.numpy().astype(np.int32).tolist(), 1, color, -1)
    
        tcrop = crop
        
        tframe = frame

        tdpp = dpp

        #viz

        for index_, crop in enumerate(crops):
            crop = cv2.resize(crop, (640, 480))
            crops[index_] = crop
        canvas = np.hstack(crops)
        

        frame = np.ndarray.astype(canvas[...,::-1], np.float32)
        # frame = cv2.putText(frame, f"{1/(time()-tic):.2f}", (50, 50) , cv2.FONT_HERSHEY_SIMPLEX ,  
        #                    1, (255, 0, 0), 2, cv2.LINE_AA)
        
        # dpp_new*=(64/fsize)
        dpp_new[0]*=(640/fsize)
        dpp_new[1]*=(480/fsize)
        dpp_new[0]+=640
        # print(dpp_new)
        cv2.circle(frame, dpp_new.astype(np.int32).tolist(), 8, (0, 255, 0), -1)
        
        # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(IPImage(data=frame.tobytes()))
        
        # print(f'{1/(time()-tic):.2f}', end='          \r')
        if cnt > 19:
            print("OVER")
        if cnt > 20:
            cnt = 0
            clear_output(wait=True)
            display_handle=IPdisplay(None, display_id=True)
except KeyboardInterrupt:
    pass
finally:
    display_handle.update(None)
#Image.fromarray(canvas[...,::-1])

None

[ 0.0598056  -0.03940021]
[0.64811145 0.71492192]
[-0.5488951  -0.20112309]
[ 3.38596381 -1.17410553]
[ 13.7591995  -10.27848719]
[ 8.14314864 -4.43207242]
[1.99610605 2.69981625]
[ 2.2273344  -0.81749945]
[-2.82252744  4.83432838]
[-2.60741308  2.13760683]
[-1.79434008  4.69256638]
[0.78470077 8.56662227]
[-5.05555492  2.80911642]
[ 1.72981707 -2.13362267]
[3.1146057  0.41752729]
[-1.60937748 -5.02864305]
[-1.49917126 -0.78063396]


# Other

In [8]:
cv2.findHomography(tracks[:,0,:].numpy().reshape(-1,1,2), tracks[:,1,:].numpy().reshape(-1,1,2), cv2.RANSAC, 5.0)

(array([[ 1.04653590e+00,  5.33360147e-03, -1.05630577e+00],
        [-4.62948390e-03,  1.05435557e+00, -1.45384428e+00],
        [-1.28266517e-04,  3.30127736e-04,  1.00000000e+00]]),
 array([[0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1]], dtype=uint8))

In [9]:
type(tracks)

torch.Tensor

In [ ]:
tracks.shape

In [10]:
canvas = tframe.copy()



In [11]:
seq = torch.cat((tcrop[None], crop[None]))[None]
seq.shape

torch.Size([1, 2, 64, 64, 3])

In [12]:
fsize = 256
tframes = torch.tensor(np.asarray(frames)).permute(0,3,1,2)
tcrops = T.CenterCrop(fsize)(tframes).float()
tcrops = preprocess_frames(tcrops)[None].to(device).permute(0,1,3,4,2)

step = 32
points = np.asarray([[0, i+step//2, j+step//2] for i in range(0, fsize, step) for j in range(0, fsize, step)])
points = torch.tensor(points)[None].float().to(device)
#points = points.float()[None]
tcrops.shape, points.shape 

NameError: name 'frames' is not defined

In [ ]:
from time import time

num_frames, height, width = tcrops.shape[1:4]

# Model inference
tic = time()
outputs = model(tcrops, points)
print(num_frames/(time()-tic))

tracks, occlusions, expected_dist = outputs['tracks'][0], outputs['occlusion'][0], outputs['expected_dist'][0]

# Binarize occlusions
visibles = postprocess_occlusions(occlusions, expected_dist)

tracks = tracks.cpu().numpy()
visibles = visibles.cpu().numpy()

In [ ]:
#!wget https://storage.googleapis.com/dm-tapnet/tapir_checkpoint_panning.pt
#!wget https://storage.googleapis.com/dm-tapnet/bootstapir_checkpoint.pt

In [ ]:
import torchvision.transforms as t


dir(t)
#help(t.CenterCrop)